# WAVES

In [1]:
import pyodbc
import pandas as pd

In [2]:
cnxn = pyodbc.connect("Driver={SQL Server Native Client 11.0};"
                        "Server=wd02;"
                        "Database=FootPrint;"
                        "Trusted_Connection=yes;")

### Query

In [3]:
df = pd.read_sql_query("SELECT CONVERT(varchar,w.releaseDate,100) Date, w.id, wa.name Warehouse, s.lookupCode Shipments, p.name Customer, ss.name ShipmentStatus\
    FROM datex_footprint.Waves w\
    JOIN datex_footprint.WaveStatuses ws\
        ON ws.id = w.statusId\
    JOIN datex_footprint.Warehouses wa\
        ON wa.id = w.warehouseId\
    JOIN datex_footprint.shipments s\
        ON s.waveId = w.id\
    JOIN datex_footprint.ShipmentOrderLookup sol\
        ON sol.shipmentId = s.id\
    JOIN datex_footprint.Orders o\
        ON o.id = sol.orderId\
    JOIN datex_footprint.Projects p\
        ON p.id = o.projectId\
    JOIN datex_footprint.ShipmentStatuses ss\
        ON ss.id = s.statusId\
    WHERE releaseDate > dateadd(day,-1,getdate())\
        AND w.statusId = 3\
    ORDER BY releaseDate DESC", cnxn)

# Create Waves Table Data

In [4]:
df.head(10)

,Date,id,Warehouse,Shipments,Customer,ShipmentStatus
0,Jun 23 2022 9:15AM,240271,10,Eme-100534,Emerginnova,Released
1,Jun 23 2022 9:15AM,240270,10,8179636,Eywa Pharma Inc.,Released
2,Jun 23 2022 9:15AM,240270,10,8179651,Eywa Pharma Inc.,Released
3,Jun 23 2022 9:15AM,240270,10,8180185,Eywa Pharma Inc.,Released
4,Jun 23 2022 9:15AM,240270,10,8180186,Eywa Pharma Inc.,Released
5,Jun 23 2022 9:15AM,240270,10,8180187,Eywa Pharma Inc.,Released
6,Jun 23 2022 9:15AM,240270,10,8180188,Eywa Pharma Inc.,Released
7,Jun 23 2022 9:15AM,240270,10,8180189,Eywa Pharma Inc.,Released
8,Jun 23 2022 9:13AM,240269,20,8182194,"Exact-Rx, Inc.",Released
9,Jun 23 2022 9:13AM,240269,20,8182195,"Exact-Rx, Inc.",Released


In [5]:
# Count Shipments per Wave and Status
count_shipments_statusdf = df.groupby(['id', 'ShipmentStatus']).count()[['Shipments']]
count_shipments_statusdf.reset_index(inplace=True)
count_shipments_statusdf.tail()

,id,ShipmentStatus,Shipments
30,240267,Released,16
31,240268,Released,1
32,240269,Released,2
33,240270,Released,7
34,240271,Released,1


In [6]:
# Convert Statuses to Columns
shipment_pivot = count_shipments_statusdf.pivot(index='id', columns='ShipmentStatus', values='Shipments')
shipment_pivot.tail()

ShipmentStatus,Completed,Executing,Released
id,,,
240267,NaN,NaN,16.0
240268,NaN,NaN,1.0
240269,NaN,NaN,2.0
240270,NaN,NaN,7.0
240271,NaN,NaN,1.0


In [7]:
# Convert NaN to 0
shipment_pivot = shipment_pivot.fillna(0)
shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0.0,0.0,1.0
240161,0.0,0.0,1.0
240194,0.0,0.0,1.0
240197,0.0,1.0,0.0
240200,1.0,10.0,0.0


In [8]:
# Convert Float to Int
try:
    shipment_pivot['Completed'] = shipment_pivot['Completed'].astype(int)
except:
    print('Column Completed does not exist yet')

try:
    shipment_pivot['Executing'] = shipment_pivot['Executing'].astype(int)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'] = shipment_pivot['Released'].astype(int)
except:
    print('Column Released does not exist yet')

shipment_pivot.head()

ShipmentStatus,Completed,Executing,Released
id,,,
240135,0,0,1
240161,0,0,1
240194,0,0,1
240197,0,1,0
240200,1,10,0


In [9]:
# Convert id index to a column
shipment_pivot.reset_index(inplace=True)
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,0,0,1
1,240161,0,0,1
2,240194,0,0,1
3,240197,0,1,0
4,240200,1,10,0


In [10]:
# Convert zeros to blanks. This also change the data type to String
try:
    shipment_pivot['Completed'].replace([0, ''], '', inplace=True)
except:
    print('Column Completed does not exist yet')
    
try:
    shipment_pivot['Executing'].replace([0, ''], '', inplace=True)
except:
    print('Column Executing does not exist yet')

try:
    shipment_pivot['Released'].replace([0, ''], '', inplace=True)
except:
    print('Column Released does not exist yet')
    
shipment_pivot.head()

ShipmentStatus,id,Completed,Executing,Released
0,240135,,,1
1,240161,,,1
2,240194,,,1
3,240197,,1,
4,240200,1,10,


In [11]:
shipment_pivot.dtypes

ShipmentStatus
id            int64
Completed    object
Executing    object
Released     object
dtype: object

In [12]:
# Change columns order
df = df[['Date', 'id', 'Warehouse', 'Customer']]

In [13]:
# Drops duplicates
df = df.drop_duplicates()

In [14]:
df.head(10)

,Date,id,Warehouse,Customer
0,Jun 23 2022 9:15AM,240271,10,Emerginnova
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.
8,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc."
10,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC"
11,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.
27,Jun 23 2022 8:57AM,240266,10,Bio-PRF
28,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation
70,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation
104,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation
139,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc."


In [15]:
# Merge df with pivot table
df2 = df.merge(shipment_pivot, left_on='id', right_on='id')
df2.head(10)

,Date,id,Warehouse,Customer,Completed,Executing,Released
0,Jun 23 2022 9:15AM,240271,10,Emerginnova,,,1
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.,,,7
2,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc.",,,2
3,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC",,,1
4,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.,,,16
5,Jun 23 2022 8:57AM,240266,10,Bio-PRF,,,1
6,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,,,42
7,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,,,34
8,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,,,35
9,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",,,1


In [16]:
wave_df = pd.DataFrame()
statuses = list(df2.columns)

# Reorder columns
if ('Released' in statuses) and ('Executing' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing', 'Completed']]
elif ('Released' in statuses) and ('Executing' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Executing']]
elif ('Released' in statuses) and ('Completed' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released', 'Completed']]
elif ('Released' in statuses):
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer', 'Released']]
else:
    wave_df = df2[['Date', 'id', 'Warehouse', 'Customer']]

In [17]:
wave_df.head(10)

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 9:15AM,240271,10,Emerginnova,1,,
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.,7,,
2,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc.",2,,
3,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC",1,,
4,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.,16,,
5,Jun 23 2022 8:57AM,240266,10,Bio-PRF,1,,
6,Jun 23 2022 8:53AM,240265,10,ISDIN Corporation,42,,
7,Jun 23 2022 8:32AM,240261,10,ISDIN Corporation,34,,
8,Jun 23 2022 8:32AM,240260,10,ISDIN Corporation,35,,
9,Jun 23 2022 8:30AM,240255,10,"Citieffe, Inc.",1,,


In [18]:
# Save Waves DF into json file
wave_df.to_json('data/waves.json')

# Warehouse Pie Chart Data

In [19]:
# Save df2 in df3
warehouse_df = wave_df
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 9:15AM,240271,10,Emerginnova,1,,
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.,7,,
2,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc.",2,,
3,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC",1,,
4,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.,16,,


In [20]:
# Conver String to Float
try:
    warehouse_df['Released'] = pd.to_numeric(warehouse_df['Released'],errors='coerce')
except:
    print('Column Released does not exist yet')

try:
    warehouse_df['Executing'] = pd.to_numeric(warehouse_df['Executing'],errors='coerce')
except:
    print('Column Executing does not exist yet')

try:
    warehouse_df['Completed'] = pd.to_numeric(warehouse_df['Completed'],errors='coerce')
except:
    print('Column Completed does not exist yet')

warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 9:15AM,240271,10,Emerginnova,1.0,NaN,NaN
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.,7.0,NaN,NaN
2,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc.",2.0,NaN,NaN
3,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC",1.0,NaN,NaN
4,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.,16.0,NaN,NaN


In [21]:
# Conver NaN to Zero
warehouse_df = warehouse_df.fillna(0)
warehouse_df.head()

,Date,id,Warehouse,Customer,Released,Executing,Completed
0,Jun 23 2022 9:15AM,240271,10,Emerginnova,1.0,0.0,0.0
1,Jun 23 2022 9:15AM,240270,10,Eywa Pharma Inc.,7.0,0.0,0.0
2,Jun 23 2022 9:13AM,240269,20,"Exact-Rx, Inc.",2.0,0.0,0.0
3,Jun 23 2022 9:12AM,240268,16,"SHL Pharma, LLC",1.0,0.0,0.0
4,Jun 23 2022 9:11AM,240267,20,Alumier Labs Inc.,16.0,0.0,0.0


In [22]:
# Group by Warehouse
warehouse_df = warehouse_df.groupby(['Warehouse']).sum()
warehouse_df

,id,Released,Executing,Completed
Warehouse,,,,
10,2642676,131.0,2.0,0.0
15,240205,0.0,1.0,0.0
16,960940,4.0,0.0,0.0
19,1441355,12.0,14.0,7.0
20,1441412,21.0,16.0,1.0
21,480408,1.0,1.0,0.0


In [23]:
# Reset Index
warehouse_df.reset_index(inplace=True)
warehouse_df

,Warehouse,id,Released,Executing,Completed
0,10,2642676,131.0,2.0,0.0
1,15,240205,0.0,1.0,0.0
2,16,960940,4.0,0.0,0.0
3,19,1441355,12.0,14.0,7.0
4,20,1441412,21.0,16.0,1.0
5,21,480408,1.0,1.0,0.0


In [24]:
warehouses = list(warehouse_df.columns)
warehouse_df2 = pd.DataFrame()

if ('Released' in warehouses) and ('Executing' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing', 'Completed']]
elif ('Released' in warehouses) and ('Executing' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Executing']]
elif ('Released' in warehouses) and ('Completed' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released', 'Completed']]
elif ('Released' in warehouses):
    warehouse_df2 = warehouse_df[['Warehouse', 'Released']]
else:
    warehouse_df2 = warehouse_df[['Warehouse']]

warehouse_df2

,Warehouse,Released,Executing,Completed
0,10,131.0,2.0,0.0
1,15,0.0,1.0,0.0
2,16,4.0,0.0,0.0
3,19,12.0,14.0,7.0
4,20,21.0,16.0,1.0
5,21,1.0,1.0,0.0


In [25]:
warehouse_df2 = pd.melt(warehouse_df2, id_vars=["Warehouse"], 
                  var_name="Status", value_name="Value")
warehouse_df2.head()

,Warehouse,Status,Value
0,10,Released,131.0
1,15,Released,0.0
2,16,Released,4.0
3,19,Released,12.0
4,20,Released,21.0


In [26]:
warehouse_df3 = warehouse_df2.pivot(index ='Status', columns='Warehouse', values='Value')
warehouse_df3

Warehouse,10,15,16,19,20,21
Status,,,,,,
Completed,0.0,0.0,0.0,7.0,1.0,0.0
Executing,2.0,1.0,0.0,14.0,16.0,1.0
Released,131.0,0.0,4.0,12.0,21.0,1.0


In [27]:
warehouse_df4 = warehouse_df3.reset_index()
warehouse_df4

Warehouse,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,7.0,1.0,0.0
1,Executing,2.0,1.0,0.0,14.0,16.0,1.0
2,Released,131.0,0.0,4.0,12.0,21.0,1.0


In [28]:
warehouse_df4 = warehouse_df4.rename_axis(None, axis=1)
warehouse_df4

,Status,10,15,16,19,20,21
0,Completed,0.0,0.0,0.0,7.0,1.0,0.0
1,Executing,2.0,1.0,0.0,14.0,16.0,1.0
2,Released,131.0,0.0,4.0,12.0,21.0,1.0


In [29]:
# Save Warehouse into JSON file
warehouse_df4.to_json('data/warehouse.json')

In [30]:
# Close connection
cnxn.close()